In [ ]:

import pandas as pd
pd.set_option("max_colwidth", None)
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import torch
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet, NaNLabelEncoder
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, QuantileLoss, MAPE
import locale
import pickle
locale.setlocale(locale.LC_ALL, 'it_IT')


In [ ]:
df_settimanale = pd.read_csv("processed_data/first_dataset.csv")
df_settimanale["time_idx"] = df_settimanale["time_idx"].astype(int)
df_settimanale["azoto_oraria_max"] = df_settimanale["azoto_oraria_max"].astype(float)
df_settimanale["zolfo_giornaliera"] = df_settimanale["zolfo_giornaliera"].astype(float)
df_settimanale["ozono_8h_max"] = df_settimanale["ozono_8h_max"].astype(float)
df_settimanale["pm2dot5_giornaliera"] = df_settimanale["pm2dot5_giornaliera"].astype(float)
df_settimanale["pm10_giornaliera"] = df_settimanale["pm10_giornaliera"].astype(float)
df_settimanale["provincia"] = df_settimanale["provincia"].astype(str)


In [ ]:
df_settimanale=df_settimanale.drop(["Unnamed: 0"], axis=1)
df_settimanale

Provo ad aumentare l'encoder length, che dovrebbe dirmi, semplificando, "quanto guardare indietro" per la previsione.
Si potrebbero inoltre aggiungere features numeriche statiche come media e deviazione standard per almeno il pm10, per ogni provincia.

In [ ]:
max_prediction_length = 4
max_encoder_length = 32
training_cutoff = df_settimanale["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df_settimanale[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="pm10_giornaliera",
    group_ids=["provincia"],
    min_encoder_length=max_encoder_length, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["provincia"],
    static_reals=[], # da agginugere media e dev_std del pm10
    time_varying_known_categoricals=[],
    time_varying_known_reals=["time_idx","relative_time_w","relative_time_y","relative_time_m"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "azoto_oraria_max","zolfo_giornaliera",
        "ozono_8h_max","pm2dot5_giornaliera","pm10_giornaliera", "impianti_rifiuti", "siti_inquinati"
    ],
    target_normalizer=GroupNormalizer(
        groups=["provincia"], transformation="relu", scale_by_group=True
    ),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [ ]:
validation = TimeSeriesDataSet.from_dataset(training, df_settimanale, predict=True, stop_randomization=True)

batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*4, num_workers=0)

Calcolo delle performance della baseline, utili per verificare i risultati dell'algoritmo

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_model = Baseline()
baseline_predictions = baseline_model.predict(val_dataloader)
maPe = mean_absolute_percentage_error(actuals, baseline_predictions)
maPe

## Training

Gli iperparametri sono ottimizzati tramite optuna. Inoltre si è visto graficamente che dopo l'epoca 67 il modello va in overfitting.
Il miglior modello ottenuto è version_6

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=15, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate

pl.seed_everything(42)
trainer = pl.Trainer(
    max_epochs=67,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.39993629374404666,
    callbacks=[lr_logger],
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.06130995353990287,
    hidden_size=117,  # most important hyperparameter apart from learning rate
    attention_head_size=4,# number of attention heads. Set to up to 4 for large datasets
    dropout=0.2,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=43,  # set to <= hidden_size
    output_size=7,  
    loss=QuantileLoss(),
    reduce_on_plateau_patience=50,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    
)

In [ ]:
import pickle

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=200,
    max_epochs=50,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

## Valutazione del modello

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)

maPe = mean_absolute_percentage_error(actuals, predictions)
maPe

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)


for idx in range(4):
    best_tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True)

In [ ]:
predictions, x = best_tft.predict(val_dataloader, return_x=True)
predictions_vs_actuals = best_tft.calculate_prediction_actual_by_variable(x, predictions)
best_tft.plot_prediction_actual_by_variable(predictions_vs_actuals)

In [ ]:
interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

## Grafici

In [ ]:
import plotly.graph_objects as go

In [ ]:
provincia = "Trieste"

raw_prediction, xraw = best_tft.predict(
    training.filter(lambda x: (x.provincia == provincia)),
    mode="prediction",
    return_x=True,
)
baseline_predictions = Baseline().predict(training.filter(lambda x: (x.provincia == provincia)),
    mode="prediction",
    return_x=False)

predicted_df = pd.DataFrame([],columns=["time_idx","values","baseline_values"])
i=0
while i < len(raw_prediction):
    for x in range(0,len(raw_prediction[i])):
        predicted_df = predicted_df.append({"time_idx":xraw["decoder_time_idx"][i][x],"values":raw_prediction[i][x].numpy(),"baseline_values":baseline_predictions[i][x].numpy()},ignore_index=True)
    i += len(raw_prediction[i])
predicted_df["values"] = predicted_df["values"].astype(float)
predicted_df["baseline_values"] = predicted_df["baseline_values"].astype(float)
predicted_df = predicted_df.groupby(by="time_idx").mean().reset_index()
target = df_settimanale[df_settimanale["provincia"]==provincia]
fig = go.Figure()
fig.add_trace(go.Scatter(x=target["time_idx"], y=target["pm10_giornaliera"],mode='lines+markers',name="Realtà"))
fig.add_trace(go.Scatter(x=predicted_df["time_idx"], y=predicted_df["baseline_values"],mode='lines+markers',name="Baseline"))
fig.add_trace(go.Scatter(x=predicted_df["time_idx"], y=predicted_df["values"],mode='lines+markers',name="Predizione"))
fig.show()

In [ ]:
provincia = "Trieste"
raw_prediction, xraw = best_tft.predict(
    validation.filter(lambda x: (x.provincia == provincia)),
    mode="prediction",
    return_x=True,
)
predicted_df = pd.DataFrame([],columns=["time_idx","values"])
i=0
while i < len(raw_prediction):
    for x in range(0,len(raw_prediction[i])):
        predicted_df = predicted_df.append({"time_idx":xraw["decoder_time_idx"][i][x],"values":raw_prediction[i][x].numpy()},ignore_index=True)
    i += len(raw_prediction[i])
predicted_df["values"] = predicted_df["values"].astype(float)
target = df_settimanale[df_settimanale["provincia"]==provincia]
fig = go.Figure()
fig.add_trace(go.Scatter(x=target["time_idx"], y=target["pm10_giornaliera"],mode='lines+markers',name="Realtà"))
fig.add_trace(go.Scatter(x=predicted_df["time_idx"], y=predicted_df["values"],mode='lines+markers',name="Predizione"))
fig.show()